# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 9:24AM,240820,20,8212470,"Exact-Rx, Inc.",Released
1,Jun 30 2022 9:24AM,240820,20,8212471,"Exact-Rx, Inc.",Released
2,Jun 30 2022 9:22AM,240818,16,SSF-8212520,Sartorius Stedim Filters Inc.,Released
3,Jun 30 2022 9:20AM,240817,10,SOIVA0000494,Ivaoes Animal Health,Released
4,Jun 30 2022 9:19AM,240816,10,SOTAH0000720,"Nextsource Biotechnology, LLC",Released
5,Jun 30 2022 9:19AM,240816,10,SONSB0001849,"Nextsource Biotechnology, LLC",Released
6,Jun 30 2022 9:19AM,240815,12,HH-35547,Hush Hush,Released
7,Jun 30 2022 9:19AM,240815,12,HH-35548,Hush Hush,Released
8,Jun 30 2022 9:19AM,240815,12,HH-35549,Hush Hush,Released
9,Jun 30 2022 9:19AM,240815,12,HH-35550,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,240815,Released,7
35,240816,Released,2
36,240817,Released,1
37,240818,Released,1
38,240820,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240815,NaN,NaN,7.0
240816,NaN,NaN,2.0
240817,NaN,NaN,1.0
240818,NaN,NaN,1.0
240820,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240745,1.0,11.0,9.0
240750,0.0,25.0,1.0
240751,0.0,0.0,1.0
240763,1.0,1.0,4.0
240766,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240745,1,11,9
240750,0,25,1
240751,0,0,1
240763,1,1,4
240766,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240745,1,11,9
1,240750,0,25,1
2,240751,0,0,1
3,240763,1,1,4
4,240766,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240745,1,11,9
1,240750,,25,1
2,240751,,,1
3,240763,1,1,4
4,240766,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc."
2,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.
3,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health
4,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC"
6,Jun 30 2022 9:19AM,240815,12,Hush Hush
13,Jun 30 2022 9:09AM,240813,18,"ClearSpec, LLC"
14,Jun 30 2022 9:08AM,240812,10,MedStone Pharma LLC
15,Jun 30 2022 9:08AM,240811,12,Hush Hush
16,Jun 30 2022 9:02AM,240809,10,Beach Products Inc
30,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",,,2
1,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.,,,1
2,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health,,,1
3,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC",,,2
4,Jun 30 2022 9:19AM,240815,12,Hush Hush,,,7
5,Jun 30 2022 9:09AM,240813,18,"ClearSpec, LLC",,,1
6,Jun 30 2022 9:08AM,240812,10,MedStone Pharma LLC,,,1
7,Jun 30 2022 9:08AM,240811,12,Hush Hush,,,1
8,Jun 30 2022 9:02AM,240809,10,Beach Products Inc,,,14
9,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc.",,1,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",2,,
1,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.,1,,
2,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health,1,,
3,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC",2,,
4,Jun 30 2022 9:19AM,240815,12,Hush Hush,7,,
5,Jun 30 2022 9:09AM,240813,18,"ClearSpec, LLC",1,,
6,Jun 30 2022 9:08AM,240812,10,MedStone Pharma LLC,1,,
7,Jun 30 2022 9:08AM,240811,12,Hush Hush,1,,
8,Jun 30 2022 9:02AM,240809,10,Beach Products Inc,14,,
9,Jun 30 2022 8:57AM,240808,12,"NBTY Global, Inc.",1,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",2,,
1,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.,1,,
2,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health,1,,
3,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC",2,,
4,Jun 30 2022 9:19AM,240815,12,Hush Hush,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",2.0,NaN,NaN
1,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health,1.0,NaN,NaN
3,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
4,Jun 30 2022 9:19AM,240815,12,Hush Hush,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",2.0,0.0,0.0
1,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health,1.0,0.0,0.0
3,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
4,Jun 30 2022 9:19AM,240815,12,Hush Hush,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1926380,42.0,9.0,0.0
12,722434,9.0,1.0,0.0
16,722424,7.0,2.0,0.0
17,240768,1.0,0.0,0.0
18,240813,1.0,0.0,0.0
19,722373,54.0,7.0,0.0
20,1444604,18.0,62.0,2.0
21,1203910,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1926380,42.0,9.0,0.0
1,12,722434,9.0,1.0,0.0
2,16,722424,7.0,2.0,0.0
3,17,240768,1.0,0.0,0.0
4,18,240813,1.0,0.0,0.0
5,19,722373,54.0,7.0,0.0
6,20,1444604,18.0,62.0,2.0
7,21,1203910,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,9.0,0.0
1,12,9.0,1.0,0.0
2,16,7.0,2.0,0.0
3,17,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,54.0,7.0,0.0
6,20,18.0,62.0,2.0
7,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,9.0
2,16,Released,7.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
Executing,9.0,1.0,2.0,0.0,0.0,7.0,62.0,1.0
Released,42.0,9.0,7.0,1.0,1.0,54.0,18.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,9.0,1.0,2.0,0.0,0.0,7.0,62.0,1.0
2,Released,42.0,9.0,7.0,1.0,1.0,54.0,18.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,9.0,1.0,2.0,0.0,0.0,7.0,62.0,1.0
2,Released,42.0,9.0,7.0,1.0,1.0,54.0,18.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()